In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 

import keras 

# Make sure that the keras version is greater than 1.2 because of tf.pack incompatibility
print(keras.__version__)

2.1.2


Using TensorFlow backend.


In [2]:
# Build the Model that will be used here 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications import xception, Xception

nb_classes = 4
image_shape = [299, 299, 3]

# Train on a pre-trained model
pre_trained = True

def get_model():
    base_model = xception.Xception(include_top=False, weights='imagenet', classes=nb_classes)
    x = base_model.output

    x = Conv2D(32, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)
    
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(512, activation='relu')(x)
    
    # and a softmax layer for classification
    predictions = Dense(nb_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
    if pre_trained:
        try:
            model = keras.models.load_model('light_classifier_model.h5')
            print("The existing weights of the model have been loaded successfully.")
        except Exception:
            print("Could not find the model file .. Check if you are using the correct name")
        
    
    # Dont train the Xception layers 
    for layer in base_model.layers:
        layer.trainable = False

  # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
    return model


batch_size = 8
nb_epoch = 30

model = get_model()
model.summary()

The existing weights of the model have been loaded successfully.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________

In [ ]:
datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.1,
                             fill_mode='nearest', rescale=1. / 255)
image_data_gen = datagen.flow_from_directory('train', target_size=(299, 299), 
                 classes=['green', 'red', 'unknown', 'yellow'], batch_size=batch_size)

model.fit_generator(image_data_gen, epochs=nb_epoch, steps_per_epoch= 128)

model.save('light_classifier_model.h5')

In [13]:
## Setting up testing 

import os
import glob

from PIL import Image
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

labels = ['green', 'red', 'unknown', 'yellow']

def reshape_image(image):
  x = img_to_array(image.resize((299, 299), Image.ANTIALIAS))
  return x[None, :]

model = load_model('light_classifier_model.h5')

print("The Current Real World Model has been successfully loaded ")

The Current Real World Model has been successfully loaded 


In [22]:
number_correct = 0
num_total = 0

# Test on Red samples 
images = glob.iglob("test/green/*.jpg")

for image in images:
    print(image)
    num_total +=1
    image = load_img(image)
    #print(model.predict(reshape_image(image)).__str__())
    index = np.argmax(model.predict(reshape_image(image)))
    #print(index)
    if index == 0:
        number_correct +=1

        
print("Total Number of Samples: {}".format(num_total))
print("Total Correct Predictions: {}".format(number_correct))
print("The accuracy with test images is {:.2f}".format(number_correct/num_total))

test/green/just_traffic_light_0281.jpg
test/green/just_traffic_light_0066.jpg
test/green/just_traffic_light_0305.jpg
test/green/just_traffic_light_0602.jpg
test/green/just_traffic_light_0054.jpg
test/green/loop_with_traffic_light_1083.jpg
test/green/just_traffic_light_0665.jpg
test/green/just_traffic_light_0691.jpg
test/green/just_traffic_light_0321.jpg
test/green/just_traffic_light_0005.jpg
test/green/just_traffic_light_0264.jpg
test/green/just_traffic_light_0009.jpg
test/green/just_traffic_light_0690.jpg
test/green/just_traffic_light_0328.jpg
test/green/loop_with_traffic_light_0713.jpg
test/green/just_traffic_light_0013.jpg
test/green/just_traffic_light_0344.jpg
test/green/just_traffic_light_0601.jpg
test/green/just_traffic_light_0275.jpg
test/green/just_traffic_light_0575.jpg
test/green/just_traffic_light_0025.jpg
test/green/just_traffic_light_0286.jpg
test/green/loop_with_traffic_light_1006.jpg
test/green/just_traffic_light_0010.jpg
test/green/just_traffic_light_0674.jpg
test/green